In [1]:
import numpy as np
from torch.nn import Conv3d
from torch import from_numpy
from numpy.testing import assert_array_equal

In [2]:
def conv3d(input, weight, bias, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1,
                   bias_enabled=True, padding_mode='zeros'):
    batches = len(input)
    out = []

    for b in range(batches):
        d_in, h_in, w_in = (input[b]).shape[1], (input[b]).shape[2], (input[b]).shape[3]

        kernel_size = (kernel_size, kernel_size, kernel_size) if not isinstance(kernel_size, tuple) else kernel_size
        stride = (stride, stride, stride) if not isinstance(stride, tuple) else stride
        dilation = (dilation, dilation, dilation) if not isinstance(dilation, tuple) else dilation

        padding = padding if isinstance(padding, tuple) else (padding, padding, padding)

        d_out = int((d_in + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) / stride[0] + 1)
        h_out = int((h_in + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) / stride[1] + 1)
        w_out = int((w_in + 2 * padding[2] - dilation[2] * (kernel_size[2] - 1) - 1) / stride[2] + 1)

        out.append(np.zeros((out_channels, d_out, h_out, w_out)))

        for c_out in range(out_channels):
            for z_out in range(d_out):
                for y_out in range(h_out):
                    for x_out in range(w_out):
                        summation = 0
                        for c_in in range(in_channels):
                            for kernel_z in range(kernel_size[0]):
                                for kernel_y in range(kernel_size[1]):
                                    for kernel_x in range(kernel_size[2]):
                                        z_in = z_out * stride[0] + kernel_z * dilation[0] - padding[0]
                                        y_in = y_out * stride[1] + kernel_y * dilation[1] - padding[1]
                                        x_in = x_out * stride[2] + kernel_x * dilation[2] - padding[2]
                                        if 0 <= z_in < d_in and 0 <= y_in < h_in and 0 <= x_in < w_in:
                                            summation += input[b][c_in][z_in][y_in][x_in] * \
                                                          weight[c_out][c_in][kernel_z][kernel_y][kernel_x]
                                        elif padding_mode == 'replicate':
                                            z_in = max(0, min(z_in, d_in - 1))
                                            y_in = max(0, min(y_in, h_in - 1))
                                            x_in = max(0, min(x_in, w_in - 1))
                                            summation += input[b][c_in][z_in][y_in][x_in] * \
                                                          weight[c_out][c_in][kernel_z][kernel_y][kernel_x]

                        out[b][c_out][z_out][y_out][x_out] = summation + (bias[c_out] if bias_enabled else 0)

    return np.array(out)

In [3]:
def deffault_data(bias = None):
    channel_1 = [[[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]],
                 [[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]],
                 [[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]],
                 [[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]]]
    channel_2 = [[[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]],
                 [[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]],
                 [[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]],
                 [[1, 2, 3, 4], [9, 10, 11, 12], [5, 6, 7, 8],[13, 14, 15, 16]]]

    weight_1 = [[[1, 1],[1, 1]],[[1, 1],[1, 1]]]
    weight_2 = [[[1, 1],[1, 1]],[[1, 1],[1, 1]]]

    input = np.array([[channel_1, channel_2]])
    weight = np.array([[weight_1, weight_2], [weight_1, weight_2]])
    bias = np.array([0, 0] if bias is None else bias)

    return input, weight, bias

In [4]:
def assert_equal_own_and_torch(own, torch, input, weight, bias):
    torch.weight.data = from_numpy(weight).float()
    torch.bias.data = from_numpy(bias).float()
    own_result = np.floor(own)
    torch_result = np.floor(torch(from_numpy(input).float()).detach().numpy())
    #print(np.array(own_result).shape)
    #print(np.array(torch_result).shape)
    print("Мой результат")
    print(own_result)
    print("Результат библиотеки")
    print(torch_result)
    assert_array_equal(own_result, torch_result)

In [5]:
def test():
    print("Zero inputs and weights")
    input, weight, bias = np.zeros((1, 1, 4, 4, 4)), np.zeros((2, 1, 2, 2, 2)), np.zeros((2,))
    own_result = conv3d(input, weight, bias, in_channels=1, out_channels=2, kernel_size=2)
    torch_result = Conv3d(in_channels=1, out_channels=2, kernel_size=2)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Default data")
    input, weight, bias = deffault_data()
    own_result = conv3d(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2)
    torch_result = Conv3d(in_channels=2, out_channels=2, kernel_size=2)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Random data")
    np.random.seed(42)
    input = np.random.rand(1, 2, 5, 5, 5)
    weight = np.random.rand(2, 2, 2, 2, 2)
    bias = np.random.rand(2)
    own_result = conv3d(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2)
    torch_result = Conv3d(in_channels=2, out_channels=2, kernel_size=2)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    input, weight, bias = deffault_data()
    
    print("Stride test:")
    own_result = conv3d(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2, stride=2)
    torch_result = Conv3d(in_channels=2, out_channels=2, kernel_size=2, stride=2)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Kernel size test:")
    own_result = conv3d(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2)
    torch_result = Conv3d(in_channels=2, out_channels=2, kernel_size=2)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Padding test:")
    own_result = conv3d(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2, padding=1)
    torch_result = Conv3d(in_channels=2, out_channels=2, kernel_size=2, padding=1)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Padding test:")
    own_result = conv3d(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2, padding=2)
    torch_result = Conv3d(in_channels=2, out_channels=2, kernel_size=2, padding=2)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Dilation test:")
    own_result = conv3d(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2, dilation=2)
    torch_result = Conv3d(in_channels=2, out_channels=2, kernel_size=2, dilation=2)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)

    print("Bias test:")
    own_result = conv3d(input, weight, bias=[1, 1], in_channels=2, out_channels=2, kernel_size=2)
    torch_result = Conv3d(in_channels=2, out_channels=2, kernel_size=2, bias=True)
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias=np.array([1,1]))

    print("Padding mode test:")
    own_result = conv3d(input, weight, bias, in_channels=2, out_channels=2, kernel_size=2, padding=1, padding_mode='zeros')
    torch_result = Conv3d(in_channels=2, out_channels=2, kernel_size=2, padding=1, padding_mode='zeros')
    assert_equal_own_and_torch(own_result, torch_result, input, weight, bias)  


In [6]:
test()

Zero inputs and weights
Мой результат
[[[[[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]

   [[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]

   [[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]]


  [[[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]

   [[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]

   [[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]]]]
Результат библиотеки
[[[[[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]

   [[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]

   [[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]]


  [[[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]

   [[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]

   [[0. 0. 0.]
    [0. 0. 0.]
    [0. 0. 0.]]]]]
Default data
Мой результат
[[[[[ 88. 104. 120.]
    [120. 136. 152.]
    [152. 168. 184.]]

   [[ 88. 104. 120.]
    [120. 136. 152.]
    [152. 168. 184.]]

   [[ 88. 104. 120.]
    [120. 136. 152.]
    [152. 168. 184.]]]


  [[[ 88. 104. 120.]
    [120. 136. 152.]
    [152. 168. 184.]]

   [[ 88. 104. 120.]
    [120. 136. 152.]
    [152. 168. 184.]]

   [[ 88. 104. 